## 圧力減衰＋LE評価領域の描画



In [ ]:
# Add 3 lines by S. Wakita (07Jun2017)
import matplotlib as mpl
#mpl.use('Cairo') #for png, ps, pdf, svg, ...
mpl.use('Agg') #for png
import numpy as np
import matplotlib.pyplot as plt
import pySALEPlot as psp
from sympy import *
import math
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.optimize import curve_fit
import os
import csv

# Open data file
model=psp.opendatfile('./Ice/jdata.dat')

# Set the distance units to cm
model.setScale('mm')

# Read the experimental data
#rtime, rexpt = np.genfromtxt('./Plotting/experimental.data', usecols=(0,1), unpack=True)
#dtime, dexpt = np.genfromtxt('./Plotting/experimental.data', usecols=(2,3), unpack=True, skip_footer=15)

Grid_size = model.inputDict['GRIDSPC']
Grid_H = model.inputDict['GRIDH'][1]
Grid_V = model.inputDict['GRIDV'][1]
CPPR    = model.inputDict['OBJRESH']
vimp    = model.inputDict['OBJVEL']
Rp      = Grid_size*CPPR
vp      = -vimp
num_impactor = model.tru[0].end
num_total = model.tracer_num
ts      = 2.*Rp/vp
dt      = -model.inputDict['DTSAVE']*ts
TEND    = -model.inputDict['TEND']
DTSAVE  = -model.inputDict['DTSAVE']
STEP_num = TEND/DTSAVE
ts_step  = 1/DTSAVE

fig = plt.figure(figsize=(10,7))
ax1 = fig.add_subplot(231)
ax2 = fig.add_subplot(232)
ax3 = fig.add_subplot(233)
ax4 = fig.add_subplot(234)
ax5 = fig.add_subplot(235)

# Figure title
fig.suptitle('Al {} m/s, ts = {} s'.format(vp,ts),fontsize=15)


step0   = model.readStep(['TrP','TrT'],0)
step1   = model.readStep(['TrP','TrT'],model.laststep)

x1 = np.where(step0.xmark <= Grid_size*1e3/2)
x2 = x1[0]
x2_1 = x2[x2  > num_impactor ]
x3 = sorted(x2_1,reverse=True)

x3x = step0.xmark[x3]
x3y = -step0.ymark[x3]
x3P = step1.TrP[x3]*1e-9
maxp1 = max(x3P)
maxP1 =round(maxp1,1)
d_r=x3y/(Rp*1e3)


x4 = np.where(step0.xmark <= Grid_size/2*1e3)
x5 = x4[0]
x6 = x5[x5  < num_impactor ]
x6x = step0.xmark[x6]
x6y = step0.ymark[x6]
x6P = step1.TrP[x6]*1e-9
x6T = step1.TrT[x6]
maxp61 = max(x6P[0:20])
maxt61 = max(x6T[0:20])
maxP61 =round(maxp61,1)
maxT61 =round(maxt61,0)
maxp62 = max(x6P)
maxt62 = max(x6T)
maxP62 =round(maxp62,1)
maxT62 =round(maxt62,0)

prelabelscale = len(str(maxp1).split('.')[0])
prelabel1 = 10**(prelabelscale)
prelabel2 = prelabel1/2

if maxp1 <= prelabel2:
        prelabel1 = prelabel2
        prelabel2 = float(prelabel2)/2

maxy1 = max(d_r)
ax1.clear()
ax1.set_yscale('log')
ax1.set_xscale('log')
ax1.plot(d_r,x3P, linewidth = 0.2,c="blue",zorder=1,label="Peak Pressure = {}".format(maxp1))
ax1.legend()
# set axis labels
ax1.set_xlabel('d/R ')
ax1.set_ylabel('Pressure [GPa]')
ax1.axhline(maxp1, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
ax1.axhline(0.038, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
#ax1.axvline(maxy1, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
# ax1.set_xlim([0.005,30])
# ax1.set_ylim([1,100])


#ax1.set_yticks([prelabel2,prelabel1,maxp1])
#ax1.set_yticklabels([prelabel2,prelabel1,' {: 5.1f} '.format(maxP1)])
# set axis limits
#ax1.set_aspect ('equal')
#ax1.tick_params(axis='y', colors=color_1)


ax2.plot(x6y, x6P, linewidth = 0.2,c="blue",zorder=1,label="Peak Pressure = {}".format(maxp61))
ax2.legend()
# set axis labels
ax2.set_xlabel('z [mm]')
ax2.set_ylabel('Pressure [GPa]')
ax2.axhline(maxp61, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
ax2.axhline(maxp62, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")

# ax2.set_yticks([prelabel2,prelabel1,maxp1,maxp2])
# ax2.set_yticklabels([prelabel2,prelabel1,' {: 5.1f} '.format(maxP1),' {: 5.1f} '.format(maxP2)])
# set axis limits
# ax2.set_xlim([0,1])
#ax5.set_aspect ('equal')
#ax5.tick_params(axis='y', colors=color_1)

pre = []
dt_r   = []
impact_point = 0
for i in np.arange(0,model.laststep-1,1):
#for i in np.arange(0,20,1):
  step   = model.readStep(['Trp','TrP'],i)
  dt_t = dt*i
  pre.append(step.Trp[0]*1e-9)
  dt_r.append(dt_t)

ax3.plot(dt_r, pre, linewidth = 0.2,c="blue",zorder=1,label="Peak Pressure = {}".format(maxp1))
ax3.legend()
# set axis labels
ax3.set_xlabel('time [s]')
ax3.set_ylabel('Pressure [MPa]')
ax3.axhline(maxp61, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
ax3.axhline(maxp62, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")


#cb1.remove()
#ax2.remove()
# Plot the pressure field
for u in range(model.tracer_numu): #loop over tracer clouds
         tstart = model.tru[u].start
         tend = model.tru[u].end
         scat1 = ax4.scatter(step0.xmark[tstart:tend],step0.ymark[tstart:tend],
                c=step1.TrP[tstart:tend]*1e-6,vmin=38,vmax=1e3,
                 cmap='viridis',s=10,linewidths=0)

cb1=fig.colorbar(scat1,ax=ax4)
cb1.set_label('Tracer Prak Pressure [MPa]')
ax4.set_xlabel('x  [mm]')
ax4.set_ylabel('z  [mm]')
ax4.set_xlim([0,8*Rp*1e3])
ax4.set_ylim([-12*Rp*1e3,3*Rp*1e3+10])


x4 = np.where(Grid_size*1e3/2 <= step0.xmark)
x5 = x4[0]
x6 = x5[num_impactor <= x5 ]
x6x = step0.xmark[x6]
x6y = -step0.ymark[x6]
x6P = step1.TrP[x6]
maxp3 = max(x6P)

#x_g = len(x6)/(Grid_H-1)
#y_g = int(len(x6)/(Grid_V-1))
x_g = len(x6)/(Grid_H)
y_g = min(249,len(x6)/(Grid_V-1))

x10 =[]

for i in np.arange(0,y_g,1):
  x7 = x6[step0.xmark[x6] <= (i+1)*Grid_size*1e3]
  x8 = x7[i*Grid_size*1e3<= step0.xmark[x7] ]
  x9 = x8
  x9P = step1.TrP[x9]

  for j in np.arange(0,len(x9)-1,1):
    if x9P[j] >= 38e+6 and x9P[j+1] <= 38e+6:
      thre_id = num_impactor+i*x_g+j
      x10.append(thre_id)
      print('top',thre_id,i,j,step0.xmark[thre_id],step0.ymark[thre_id])
    if x9P[j] <= 38e+6 and x9P[j+1] >= 38e+6:
      thre_id = num_impactor+i*x_g+j
      x10.append(thre_id)
      print('bottom',thre_id,i,j,step0.xmark[thre_id],step0.ymark[thre_id])


ax4.plot(step0.xmark[x10],step0.ymark[x10], "ko", markersize=5)
ecx= step0.xmark[x10]
ecy= -step0.ymark[x10]


def ec_f(x,a,b,r):
        return b*np.sqrt(r**2-(x/a)**2)
ec_init = np.array([50,1,1])


def fit(func, x, param_init):
        X = ecx
        Y = x
        popt,pocv=curve_fit(func, X, Y, p0=param_init,maxfev=5000)
        perr = np.sqrt(np.diag(pocv))
        y=func(ecx, *popt)
        return y, popt, perr

ec_est = fit(ec_f,ecy,ec_init)

ax5.plot(ecx, ec_est[0],'r--',linewidth=1, linestyle = "dashed",label="Regression")
ax5.plot(ecx,ecy,"ko")

#ax5.legend(loc="upper right", bbox_to_anchor=(0.99, 0.99), borderaxespad=0, fontsize=10)
ax5.text(0.98,0.02,"(x/a)**2 + (y/b)**2 = r**2 \n a= {} $\pm$ {}\nb= {} $\pm$ {}\nr= {} $\pm$ {}".format(round(ec_est[1][0],2),round(ec_est[2][0],2),round(ec_est[1][1],2),round(ec_est[2][1],2),round(ec_est[1][2],2),round(ec_est[2][2],2)),color="g", fontsize=10, va='bottom', ha='right',transform=ax3.transAxes)



#fig.tight_layout(rect=[0,0,0.98,0.9])
# Save the figure
cwd = os.getcwd()
dirname = os.path.basename(cwd)

plt.savefig('../property_iso/{}isobaric.png'.format(dirname))

# 圧力減衰＋LE評価領域の描画（プロットがずれる場合）+圧力減衰の比較

In [ ]:
# Add 3 lines by S. Wakita (07Jun2017)
import matplotlib as mpl
#mpl.use('Cairo') #for png, ps, pdf, svg, ...
mpl.use('Agg') #for png
import numpy as np
import matplotlib.pyplot as plt
import pySALEPlot as psp
from sympy import *
import math
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.optimize import curve_fit
import os
import csv

# Open data file
model=psp.opendatfile('./Ice/jdata.dat')

# Set the distance units to cm
model.setScale('mm')

# Read the experimental data
#rtime, rexpt = np.genfromtxt('./Plotting/experimental.data', usecols=(0,1), unpack=True)
#dtime, dexpt = np.genfromtxt('./Plotting/experimental.data', usecols=(2,3), unpack=True, skip_footer=15)

Grid_size = model.inputDict['GRIDSPC']
Grid_H = model.inputDict['GRIDH'][1]
Grid_V = model.inputDict['GRIDV'][1]
CPPR    = model.inputDict['OBJRESH']
vimp    = model.inputDict['OBJVEL']
Rp      = Grid_size*CPPR
vp      = -vimp
num_impactor = model.tru[0].end
num_total = model.tracer_num
ts      = 2.*Rp/vp
dt      = -model.inputDict['DTSAVE']*ts
TEND    = -model.inputDict['TEND']
DTSAVE  = -model.inputDict['DTSAVE']
STEP_num = TEND/DTSAVE
ts_step  = 1/DTSAVE

fig = plt.figure(figsize=(12,7))
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)


# Figure title
fig.suptitle('Al {} m/s, ts = {} s'.format(vp,ts),fontsize=15)

#read No. 1
step0   = model.readStep(['TrP','TrT'],0)
step1   = model.readStep(['TrP','TrT'],model.laststep)

x1 = np.where(step0.xmark <= Grid_size*1e3/2)
x2 = x1[0]
x2_1 = x2[x2  > num_impactor ]
x3 = sorted(x2_1,reverse=True)

x3x = step0.xmark[x3]
x3y = -step0.ymark[x3]
x3P = step1.TrP[x3]*1e-9
maxp1 = max(x3P)
maxP1 =round(maxp1,1)
d_r=x3y/(Rp*1e3)

#read No. 2
model2=psp.opendatfile('../KatoIce14-2/Ice/jdata.dat')
model2.setScale('mm')
CPPR2    = model2.inputDict['OBJRESH']
Grid_size2 = model2.inputDict['GRIDSPC']
Rp2      = Grid_size2*CPPR2
num_impactor_2 = model2.tru[0].end
num_total_2 = model2.tracer_num

step0_2   = model2.readStep(['TrP','TrT'],0)
step1_2   = model2.readStep(['TrP','TrT'],model2.laststep)

x1_2 = np.where(step0_2.xmark <= Grid_size2*1e3/2)
x2_2 = x1_2[0]
x2_1_2 = x2_2[x2_2  > num_impactor_2 ]
x3_2 = sorted(x2_1_2,reverse=True)

x3x_2 = step0_2.xmark[x3_2]
x3y_2 = -step0_2.ymark[x3_2]
x3P_2 = step1_2.TrP[x3_2]*1e-9
maxp1_2 = max(x3P_2)
maxP1_2 =round(maxp1_2,1)
d_r_2=x3y_2/(Rp2*1e3)

#read No. 3
model3=psp.opendatfile('../KatoIce14-3/Ice/jdata.dat')
model3.setScale('mm')
CPPR3    = model3.inputDict['OBJRESH']
Grid_size3 = model3.inputDict['GRIDSPC']
Rp3      = Grid_size3*CPPR3
num_impactor_3 = model3.tru[0].end
num_total_3 = model3.tracer_num

step0_3   = model3.readStep(['TrP','TrT'],0)
step1_3   = model3.readStep(['TrP','TrT'],model3.laststep)

x1_3 = np.where(step0_3.xmark <= Grid_size3*1e3/2)
x2_3 = x1_3[0]
x2_1_3 = x2_3[x2_3  > num_impactor_3 ]
x3_3 = sorted(x2_1_3,reverse=True)

x3x_3 = step0_3.xmark[x3_3]
x3y_3 = -step0_3.ymark[x3_3]
x3P_3 = step1_3.TrP[x3_3]*1e-9
maxp1_3 = max(x3P_3)
maxP1_3 =round(maxp1_3,1)
d_r_3=x3y_3/(Rp3*1e3)



x4 = np.where(step0.xmark <= Grid_size/2*1e3)
x5 = x4[0]
x6 = x5[x5  < num_impactor ]
x6x = step0.xmark[x6]
x6y = step0.ymark[x6]
x6P = step1.TrP[x6]*1e-9
x6T = step1.TrT[x6]
maxp61 = max(x6P[0:20])
maxt61 = max(x6T[0:20])
maxP61 =round(maxp61,1)
maxT61 =round(maxt61,0)
maxp62 = max(x6P)
maxt62 = max(x6T)
maxP62 =round(maxp62,1)
maxT62 =round(maxt62,0)

prelabelscale = len(str(maxp1).split('.')[0])
prelabel1 = 10**(prelabelscale)
prelabel2 = prelabel1/2

if maxp1 <= prelabel2:
        prelabel1 = prelabel2
        prelabel2 = float(prelabel2)/2

maxy1 = max(d_r)
ax1.clear()
ax1.set_yscale('log')
ax1.set_xscale('log')
ax1.plot(d_r_2,x3P_3, linewidth = 1,c="red",zorder=1,label="aspect = 18:7   /LE = 1.17e4 [J]")
ax1.plot(d_r,x3P, linewidth = 1,c="blue",zorder=1,label="aspect = 15:10/LE = 6.39e3 [J]")
ax1.plot(d_r_2,x3P_2, linewidth = 1,c="green",zorder=1,label="aspect = 10:22/LE = 2.07e3 [J]")
ax1.legend()
# set axis labels
ax1.set_xlabel('d/R ')
ax1.set_ylabel('Pressure [GPa]')
ax1.axhline(maxp1, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
ax1.axhline(0.038, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
#ax1.axvline(maxy1, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
# ax1.set_xlim([0.005,30])
# ax1.set_ylim([1,100])


#ax1.set_yticks([prelabel2,prelabel1,maxp1])
#ax1.set_yticklabels([prelabel2,prelabel1,' {: 5.1f} '.format(maxP1)])
# set axis limits
#ax1.set_aspect ('equal')
#ax1.tick_params(axis='y', colors=color_1)


ax2.plot(x6y, x6P, linewidth = 0.2,c="blue",zorder=1,label="Peak Pressure = {}".format(maxp61))
ax2.legend()
# set axis labels
ax2.set_xlabel('z [mm]')
ax2.set_ylabel('Pressure [GPa]')
ax2.axhline(maxp61, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
ax2.axhline(maxp62, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")

# ax2.set_yticks([prelabel2,prelabel1,maxp1,maxp2])
# ax2.set_yticklabels([prelabel2,prelabel1,' {: 5.1f} '.format(maxP1),' {: 5.1f} '.format(maxP2)])
# set axis limits
# ax2.set_xlim([0,1])
#ax5.set_aspect ('equal')
#ax5.tick_params(axis='y', colors=color_1)

pre = []
dt_r   = []
impact_point = 0
for i in np.arange(0,model.laststep-1,1):
#for i in np.arange(0,20,1):
  step   = model.readStep(['Trp','TrP'],i)
  dt_t = dt*i
  pre.append(step.Trp[0]*1e-9)
  dt_r.append(dt_t)

ax3.plot(dt_r, pre, linewidth = 0.2,c="blue",zorder=1,label="Peak Pressure = {}".format(maxp1))
ax3.legend()
# set axis labels
ax3.set_xlabel('time [s]')
ax3.set_ylabel('Pressure [MPa]')
ax3.axhline(maxp61, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
ax3.axhline(maxp62, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")


#cb1.remove()
#ax2.remove()
# Plot the pressure field
for u in range(model.tracer_numu): #loop over tracer clouds
         tstart = model.tru[u].start
         tend = model.tru[u].end
         scat1 = ax4.scatter(step0.xmark[tstart:tend],step0.ymark[tstart:tend],
                c=step1.TrP[tstart:tend]*1e-6,vmin=38,vmax=1e+3,
                 cmap='viridis',s=10,linewidths=0)

cb1=fig.colorbar(scat1,ax=ax4)
cb1.set_label('Tracer Prak Pressure [MPa]')
ax4.set_xlabel('x  [mm]')
ax4.set_ylabel('z  [mm]')
ax4.set_xlim([0,8*Rp*1e3])
ax4.set_ylim([-12*Rp*1e3,3*Rp*1e3])


x4 = np.where(Grid_size*1e3/2 <= step0.xmark)
x5 = x4[0]
x6 = x5[num_impactor <= x5 ]
x6x = step0.xmark[x6]
x6y = -step0.ymark[x6]
x6P = step1.TrP[x6]
maxp3 = max(x6P)

x_g = len(x6)/(Grid_H-1)
y_g = int(len(x6)/(Grid_V-1))

y_range = (x_g-CPPR*10)
x_range = (CPPR*10)
x10 =[]

for i in np.arange(0,x_range,1):
  x7 = x6[step0.xmark[x6] <= (i+1)*Grid_size*1e3]
  x8 = x7[i*Grid_size*1e3<= step0.xmark[x7] ]
  x9 = x8[y_range:]
  x9P = step1.TrP[x9]

  for j in np.arange(0,len(x9)-1,1):
    if x9P[j] <= 38e+6 and x9P[j+1] >= 38e+6:
      thre_id = num_impactor+i*x_g+j+y_range
      x10.append(thre_id)
      print('bottom',thre_id,i,j,step0.xmark[thre_id],step0.ymark[thre_id])
    if x9P[j] >= 38e+6 and x9P[j+1] <= 38e+6:
      thre_id = num_impactor+i*x_g+j+y_range
      x10.append(thre_id)
      print('top',thre_id,i,j,step0.xmark[thre_id],step0.ymark[thre_id])

ax4.plot(step0.xmark[x10],step0.ymark[x10], "ko", markersize=1)

#fig.tight_layout(rect=[0,0,0.98,0.9])
# Save the figure
cwd = os.getcwd()
dirname = os.path.basename(cwd)

plt.savefig('../property_iso/{}isobaric.png'.format(dirname))

# 温度減衰のグラフ

In [ ]:
# Add 3 lines by S. Wakita (07Jun2017)
import matplotlib as mpl
#mpl.use('Cairo') #for png, ps, pdf, svg, ...
mpl.use('Agg') #for png
import numpy as np
import matplotlib.pyplot as plt
import pySALEPlot as psp
from sympy import *
import math
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.optimize import curve_fit
import os
import csv

# Open data file
model=psp.opendatfile('./Ice/jdata.dat')

# Set the distance units to cm
model.setScale('m')

# Read the experimental data
#rtime, rexpt = np.genfromtxt('./Plotting/experimental.data', usecols=(0,1), unpack=True)
#dtime, dexpt = np.genfromtxt('./Plotting/experimental.data', usecols=(2,3), unpack=True, skip_footer=15)

Grid_size = model.inputDict['GRIDSPC']
Grid_H = model.inputDict['GRIDH'][1]
Grid_V = model.inputDict['GRIDV'][1]
CPPR    = model.inputDict['OBJRESH']
vimp    = model.inputDict['OBJVEL']
Rp      = Grid_size*CPPR
vp      = -vimp
num_impactor = model.tru[0].end
num_total = model.tracer_num
ts      = 2.*Rp/vp
dt      = -model.inputDict['DTSAVE']*ts
TEND    = -model.inputDict['TEND']
DTSAVE  = -model.inputDict['DTSAVE']
STEP_num = TEND/DTSAVE
ts_step  = 1/DTSAVE

fig = plt.figure(figsize=(15,10))
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

# Figure title
fig.suptitle('Al {} m/s, ts = {} s'.format(vp,ts),fontsize=15)


step0   = model.readStep(['TrP','TrT'],0)
step1   = model.readStep(['TrP','TrT'],model.laststep)

x1 = np.where(step0.xmark <= Grid_size*1e3/2)
x2 = x1[0]
x2_1 = x2[x2  > num_impactor ]
x3 = sorted(x2_1,reverse=True)

x3x = step0.xmark[x3]
x3y = -step0.ymark[x3]
x3P = step1.TrP[x3]*1e-9
x3T = step1.TrT[x3]
maxp1 = max(x3P)
maxt1 = max(x3T)
maxP1 =round(maxp1,1)
maxT1 =round(maxt1,1)
d_r=x3y/(Rp*1e3)


x4 = np.where(step0.xmark <= Grid_size/2)
x5 = x4[0]
x6 = x5[x5  < num_impactor ]
x6x = step0.xmark[x6]
x6y = step0.ymark[x6]
x6P = step1.TrP[x6]*1e-9
x6T = step1.TrT[x6]
maxp61 = max(x6P[0:20])
maxt61 = max(x6T[0:20])
maxP61 =round(maxp61,1)
maxT61 =round(maxt61,0)
maxp62 = max(x6P)
maxt62 = max(x6T)
maxP62 =round(maxp62,1)
maxT62 =round(maxt62,0)


tmp = []
tmp_rear = []
dt_r   = []

tmin = []
dt_r   = []
x_im = range(0,num_impactor,1)
x_im_rear = x6[len(x6)-1]

tmp_rear_max = step1.TrT[x_im_rear]


for i in np.arange(0,model.laststep-1,1):
#for i in np.arange(0,20,1):
  step   = model.readStep(['Trt','TrT'],i)
  dt_t = dt*i
  t_im_1 = step.Trt[x_im]
  t_im_2 = []
  for j in np.arange(0,len(t_im_1)-1,1):
    if t_im_1[j] >= 1 :
      t_im_2.append(t_im_1[j])
  tmin_i = min(t_im_2)
  tmin.append(tmin_i)
  if i ==1:
    tmp.append(step.TrT[0])
  else:
    tmp.append(step.Trt[0])
  tmp_rear.append(step.Trt[x_im_rear])
  dt_r.append(dt_t)

for k in np.arange(0,model.laststep-1,1):
  if tmp_rear[k] == max(tmp_rear) :
    tmp_rear[k]=tmp_rear_max
    break

ax1.set_yscale('log')
ax1.plot(dt_r, tmp, linewidth = 0.2,c="blue",zorder=1,label="Peak Temperature = {}".format(maxt61))
ax1.legend()
# set axis labels
ax1.set_xlabel('time [s]')
ax1.set_xlim([0,0.1])
ax1.set_ylabel('Temperature [K]')
ax1.axhline(500, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")


ax2.plot(x6y, x6T, linewidth = 0.2,c="blue",zorder=1,label="Peak Temperature = {}".format(maxt61))
ax2.legend()
# set axis labels
ax2.set_xlabel('z [mm]')
ax2.set_ylabel('Temperature [K]')
ax2.axhline(maxt61, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
ax2.axhline(maxt62, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
ax2.axhline(500, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")


# ax2.set_yticks([prelabel2,prelabel1,maxp1,maxp2])
# ax2.set_yticklabels([prelabel2,prelabel1,' {: 5.1f} '.format(maxP1),' {: 5.1f} '.format(maxP2)])
# set axis limits
# ax2.set_xlim([0,1])

ax3.plot(dt_r, tmin, linewidth = 0.2,c="blue",zorder=1,label="Peak Tempareture = {}".format(max(tmin)))
ax3.legend()
# set axis labels
ax3.set_xlabel('time [s]')
ax3.set_ylabel('Temperature [K]')
ax3.axhline(max(tmin), color = "black", alpha = 0.2, lw=1, linestyle = "dashed")
ax3.axhline(500, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")

# ax4.set_yscale('log')
# ax4.plot(dt_r, tmp_rear, linewidth = 0.2,c="blue",zorder=1,label="Peak Temperature = {}".format(tmp_rear_max))
# ax4.legend()
# # set axis labels
# ax4.set_xlabel('time [s]')
# ax4.set_xlim([0,0.01])
# ax4.set_ylabel('Temperature [K]')
# ax4.axhline(500, color = "black", alpha = 0.2, lw=1, linestyle = "dashed")


#cb1.remove()
#ax2.remove()
#Plot the pressure field
for u in range(model.tracer_numu): #loop over tracer clouds
         tstart = model.tru[u].start
         tend = model.tru[u].end
         scat1 = ax4.scatter(step0.xmark[tstart:tend],step0.ymark[tstart:tend],
                 c=step1.TrT[tstart:tend],vmin=500,vmax=5000,
                 cmap='viridis',s=10,linewidths=0)

cb1=fig.colorbar(scat1,ax=ax4)
cb1.set_label('Tracer Prak Temperature [K]')
ax4.set_xlabel('x  [m]')
ax4.set_ylabel('z  [m]')
ax4.set_xlim([0,5*Rp])
ax4.set_ylim([-3*Rp,3*Rp])


#fig.tight_layout(rect=[0,0,0.98,0.9])
# Save the figure
cwd = os.getcwd()
dirname = os.path.basename(cwd)

plt.savefig('../property_iso/{}isobaric.png'.format(dirname))

# LE描画領域のみ

In [ ]:
# Add 3 lines by S. Wakita (07Jun2017)
import matplotlib as mpl
#mpl.use('Cairo') #for png, ps, pdf, svg, ...
mpl.use('Agg') #for png
import numpy as np
import matplotlib.pyplot as plt
import pySALEPlot as psp
from sympy import *
import math
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.optimize import curve_fit
import os
import csv

# Open data file
model=psp.opendatfile('./Ice/jdata.dat')

# Set the distance units to cm
model.setScale('mm')

# Read the experimental data
#rtime, rexpt = np.genfromtxt('./Plotting/experimental.data', usecols=(0,1), unpack=True)
#dtime, dexpt = np.genfromtxt('./Plotting/experimental.data', usecols=(2,3), unpack=True, skip_footer=15)
#m unit
Grid_size = model.inputDict['GRIDSPC']
Grid_H = model.inputDict['GRIDH'][1]
Highrez_V = model.inputDict['GRIDV'][1]
extend_V = model.inputDict['GRIDV'][0]
Target_depth =  model.inputDict['LAYPOS']
Grid_V = Target_depth - extend_V

if 'OBJRESV' in model.inputDict:
  print("OBJRESV exists in model.inputDict")
  CPPR    = model.inputDict['OBJRESV']
else:
  print("OBJRESV does not exist in model.inputDict")
  CPPR    = model.inputDict['OBJRESH']

vimp    = model.inputDict['OBJVEL']
Rp      = Grid_size*CPPR
vp      = -vimp
ts      = 2.*Rp/vp
num_impactor = model.tru[0].end
num_total = model.tracer_num
dt      = -model.inputDict['DTSAVE']*ts
TEND    = -model.inputDict['TEND']
DTSAVE  = -model.inputDict['DTSAVE']
STEP_num = TEND/DTSAVE
ts_step  = 1/DTSAVE


fig = plt.figure(figsize=(10,8))
ax4 = fig.add_subplot(111)

#read No. 1
step0   = model.readStep(['TrP','TrT'],0)
step1   = model.readStep(['TrP','TrT','Trd','Dam'],model.laststep)

x1 = np.where(step0.xmark <= Grid_size*1e3/2)
x2 = x1[0]
x2_1 = x2[x2  > num_impactor ]
x3 = sorted(x2_1,reverse=True)

x3x = step0.xmark[x3]
x3y = -step0.ymark[x3]
x3P = step1.TrP[x3]*1e-9
maxp1 = max(x3P)
maxP1 =round(maxp1,1)
d_r=x3y/(Rp*1e3)

x4 = np.where(step0.xmark <= Grid_size/2*1e3)
x5 = x4[0]
x6 = x5[x5  < num_impactor ]
x6x = step0.xmark[x6]
x6y = step0.ymark[x6]
x6P = step1.TrP[x6]*1e-9
x6T = step1.TrT[x6]
maxp61 = max(x6P[0:20])
maxt61 = max(x6T[0:20])
maxP61 =round(maxp61,1)
maxT61 =round(maxt61,0)
maxp62 = max(x6P)
maxt62 = max(x6T)
maxP62 =round(maxp62,1)
maxT62 =round(maxt62,0)

#cb1.remove()
#ax2.remove(

x4 = np.where(Grid_size*1e3/2 <= step0.xmark)
x5 = x4[0]
x6 = x5[num_impactor <= x5 ]
x6x = step0.xmark[x6]
x6y = -step0.ymark[x6]
x6P = step1.TrP[x6]
x6d = step1.Trd[x6]
maxp3 = max(x6P)

y_g = len(x6)/(Grid_H)
x_g = len(x6)/(Grid_V-1)

LE_shear = 0
LE_comp = 0
LE_tensile = 0
LE_ave = 0
LE_iso_2 = 0
pre_iso = []
x10 = []
x11 = []
thre_ids = []
Grid_A = Grid_size**2

for i in np.arange(0,x_g,1):
  x7 = x6[step0.xmark[x6] <= (i+1)*Grid_size*1e3]
  x8 = x7[i*Grid_size*1e3<= step0.xmark[x7] ]
  x9 = x8
  x9P = step1.TrP[x9]
  # print(LE_shear)
  for j in np.arange(0,len(x9)-1,1):
    if x9P[j] >= 38e+6:
      LE_i = x9P[j]*Grid_A*2*np.pi*((i+1/2)*Grid_size)
      LE_shear = LE_shear + LE_i

for u in np.arange(0,len(x6),1): #loop over tracer clouds
  if x6P[u] >= 38e+6:
      thre_id = num_impactor + u
      x10.append(thre_id)

Region = model.inputDict['GRIDV']
Region_num = Region[0] + Region[1]
Target = model.inputDict['GRIDH']
Target_num = Target[1]+Target[2]

Dam_x = []
Dam_y = []

# for i in np.arange(0,Target_num,1):
#   for j in np.arange(0,Region_num,1):
#     if step1.Dam[i,j] >= 0.5:
#       tracer_id = step1.findTracer(model.x[i,j],model.y[i,j])
#       if isinstance(tracer_id, list):
#         tracer_id = tracer_id[0]
#       if tracer_id >= num_impactor:
#         Dam_x.append(step1.xmark[tracer_id])
#         Dam_y.append(step1.ymark[tracer_id])
#       break


    # if x9P[j] <= 38e+6 and x9P[j+1] >= 38e+6:
    #   thre_id = num_impactor + i*y_g + j
    #   x10.append(thre_id)
    #   print('bottom',thre_id,i,j,step0.xmark[thre_id],step0.ymark[thre_id])
    # if x9P[j] >= 38e+6 and x9P[j+1] <= 38e+6:
    #   thre_id3 = num_impactor + i*y_g + j
    #   x10.append(thre_id)
    #   print('top',thre_id,i,j,step0.xmark[thre_id],step0.ymark[thre_id])

# Plot the pressure field
# for u in range(model.tracer_numu): #loop over tracer clouds
for u in range(1): #loop over tracer clouds
         tstart = model.tru[u].start
         tend = model.tru[u].end
         scat1 = ax4.scatter(step0.xmark[tstart:tend],step0.ymark[tstart:tend],
                c=step1.TrP[tstart:tend]*1e-6,vmin=10,vmax=5e+2,
                 cmap='viridis',s=40,linewidths=0)

cb1=fig.colorbar(scat1,ax=ax4)
cb1.set_label('Effective Excavation Region Pressure [MPa]')
ax4.set_xlabel('x  [mm]')
ax4.set_ylabel('z  [mm]')
ax4.set_xlim([0,75])
ax4.set_ylim([-60,15])

ax4.scatter(step0.xmark[x10],step0.ymark[x10],
                c=step1.TrP[x10]*1e-6,vmin=10,vmax=1e+3,
                 cmap='viridis',s=40,linewidths=0)
ax4.axhline(0, color = "black", linewidth=0.8)
ax4.scatter(step1.xmark[x6],step1.ymark[x6],c='lightblue',s=40,linewidths=0)


# ax4.pcolormesh(model.x,model.y,step1.Dam,vmin=0.13,vmax=1)

#fig.tight_layout(rect=[0,0,0.98,0.9])
# Save the figure
cwd = os.getcwd()
dirname = os.path.basename(cwd)

plt.savefig('../property_iso/{}isobaric_2.png'.format(dirname))

In [ ]:
# Add 3 lines by S. Wakita (07Jun2017)
import matplotlib as mpl
#mpl.use('Cairo') #for png, ps, pdf, svg, ...
mpl.use('Agg') #for png
import numpy as np
import matplotlib.pyplot as plt
import pySALEPlot as psp
from sympy import *
import math
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.optimize import curve_fit
import os
import csv

fig = plt.figure(figsize=(16,8))
ax4 = fig.add_subplot(121)
ax5 = fig.add_subplot(122)

# Open data file
model=psp.opendatfile('../aliceTILLO2/Ice/jdata.dat')

# Set the distance units to cm
model.setScale('mm')

# Read the experimental data
#rtime, rexpt = np.genfromtxt('./Plotting/experimental.data', usecols=(0,1), unpack=True)
#dtime, dexpt = np.genfromtxt('./Plotting/experimental.data', usecols=(2,3), unpack=True, skip_footer=15)
#m unit
Grid_size = model.inputDict['GRIDSPC']
Grid_H = model.inputDict['GRIDH'][1]
Highrez_V = model.inputDict['GRIDV'][1]
extend_V = model.inputDict['GRIDV'][0]
Target_depth =  model.inputDict['LAYPOS']
Grid_V = Target_depth - extend_V

if 'OBJRESV' in model.inputDict:
  print("OBJRESV exists in model.inputDict")
  CPPR    = model.inputDict['OBJRESV']
else:
  print("OBJRESV does not exist in model.inputDict")
  CPPR    = model.inputDict['OBJRESH']

vimp    = model.inputDict['OBJVEL']
Rp      = Grid_size*CPPR
vp      = -vimp
ts      = 2.*Rp/vp
num_impactor = model.tru[0].end
num_total = model.tracer_num
dt      = -model.inputDict['DTSAVE']*ts
TEND    = -model.inputDict['TEND']
DTSAVE  = -model.inputDict['DTSAVE']
STEP_num = TEND/DTSAVE
ts_step  = 1/DTSAVE


#read No. 1
step0   = model.readStep(['TrP','TrT'],0)
step1   = model.readStep(['TrP','TrT','Trd','Dam'],model.laststep)

x1 = np.where(step0.xmark <= Grid_size*1e3/2)
x2 = x1[0]
x2_1 = x2[x2  > num_impactor ]
x3 = sorted(x2_1,reverse=True)

x3x = step0.xmark[x3]
x3y = -step0.ymark[x3]
x3P = step1.TrP[x3]*1e-9
maxp1 = max(x3P)
maxP1 =round(maxp1,1)
d_r=x3y/(Rp*1e3)

x4 = np.where(step0.xmark <= Grid_size/2*1e3)
x5 = x4[0]
x6 = x5[x5  < num_impactor ]
x6x = step0.xmark[x6]
x6y = step0.ymark[x6]
x6P = step1.TrP[x6]*1e-9
x6T = step1.TrT[x6]
maxp61 = max(x6P[0:20])
maxt61 = max(x6T[0:20])
maxP61 =round(maxp61,1)
maxT61 =round(maxt61,0)
maxp62 = max(x6P)
maxt62 = max(x6T)
maxP62 =round(maxp62,1)
maxT62 =round(maxt62,0)

#cb1.remove()
#ax2.remove(

x4 = np.where(Grid_size*1e3/2 <= step0.xmark)
x5 = x4[0]
x6 = x5[num_impactor <= x5 ]
x6x = step0.xmark[x6]
x6y = -step0.ymark[x6]
x6P = step1.TrP[x6]
x6d = step1.Trd[x6]
maxp3 = max(x6P)

y_g = len(x6)/(Grid_H)
x_g = len(x6)/(Grid_V-1)

LE_shear = 0
LE_comp = 0
LE_tensile = 0
LE_ave = 0
LE_iso_2 = 0
pre_iso = []
x10 = []
x11 = []
thre_ids = []
Grid_A = Grid_size**2

for i in np.arange(0,x_g,1):
  x7 = x6[step0.xmark[x6] <= (i+1)*Grid_size*1e3]
  x8 = x7[i*Grid_size*1e3<= step0.xmark[x7] ]
  x9 = x8
  x9P = step1.TrP[x9]
  # print(LE_shear)
  for j in np.arange(0,len(x9)-1,1):
    if x9P[j] >= 38e+6:
      LE_i = x9P[j]*Grid_A*2*np.pi*((i+1/2)*Grid_size)
      LE_shear = LE_shear + LE_i

for u in np.arange(0,len(x6),1): #loop over tracer clouds
  if x6P[u] >= 38e+6:
      thre_id = num_impactor + u
      x10.append(thre_id)

Region = model.inputDict['GRIDV']
Region_num = Region[0] + Region[1]
Target = model.inputDict['GRIDH']
Target_num = Target[1]+Target[2]

Dam_x = []
Dam_y = []

# for i in np.arange(0,Target_num,1):
#   for j in np.arange(0,Region_num,1):
#     if step1.Dam[i,j] >= 0.5:
#       tracer_id = step1.findTracer(model.x[i,j],model.y[i,j])
#       if isinstance(tracer_id, list):
#         tracer_id = tracer_id[0]
#       if tracer_id >= num_impactor:
#         Dam_x.append(step1.xmark[tracer_id])
#         Dam_y.append(step1.ymark[tracer_id])
#       break


    # if x9P[j] <= 38e+6 and x9P[j+1] >= 38e+6:
    #   thre_id = num_impactor + i*y_g + j
    #   x10.append(thre_id)
    #   print('bottom',thre_id,i,j,step0.xmark[thre_id],step0.ymark[thre_id])
    # if x9P[j] >= 38e+6 and x9P[j+1] <= 38e+6:
    #   thre_id3 = num_impactor + i*y_g + j
    #   x10.append(thre_id)
    #   print('top',thre_id,i,j,step0.xmark[thre_id],step0.ymark[thre_id])

# Plot the pressure field
# for u in range(model.tracer_numu): #loop over tracer clouds
for u in range(1): #loop over tracer clouds
         tstart = model.tru[u].start
         tend = model.tru[u].end
         scat1 = ax4.scatter(step0.xmark[tstart:tend],step0.ymark[tstart:tend],
                c=step1.TrP[tstart:tend]*1e-6,vmin=10,vmax=5e+2,
                 cmap='viridis',s=40,linewidths=0)

# cb1=fig.colorbar(scat1,ax=ax4)
# cb1.set_label('Effective Excavation Region Pressure [MPa]')
ax4.set_xlabel('x  [mm]')
ax4.set_ylabel('z  [mm]')
ax4.set_xlim([0,20])
ax4.set_ylim([-15,5])

ax4.scatter(step1.xmark[x6],step1.ymark[x6],c='lightblue',s=40,linewidths=0)
ax4.scatter(step0.xmark[x10],step0.ymark[x10],
                c=step1.TrP[x10]*1e-6,vmin=10,vmax=5e+2,
                 cmap='viridis',s=40,linewidths=0)
ax4.axhline(0, color = "black", linewidth=0.8)


# ax4.pcolormesh(model.x,model.y,step1.Dam,vmin=0.13,vmax=1)

#fig.tight_layout(rect=[0,0,0.98,0.9])
# Save the figure
cwd = os.getcwd()
dirname = os.path.basename(cwd)

plt.savefig('../property_iso/{}isobaric_2.png'.format(dirname))


# Add 3 lines by S. Wakita (07Jun2017)
import matplotlib as mpl
#mpl.use('Cairo') #for png, ps, pdf, svg, ...
mpl.use('Agg') #for png
import numpy as np
import matplotlib.pyplot as plt
import pySALEPlot as psp
from sympy import *
import math
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.optimize import curve_fit
import os
import csv

# Open data file
model=psp.opendatfile('./Ice/jdata.dat')

# Set the distance units to cm
model.setScale('mm')

# Read the experimental data
#rtime, rexpt = np.genfromtxt('./Plotting/experimental.data', usecols=(0,1), unpack=True)
#dtime, dexpt = np.genfromtxt('./Plotting/experimental.data', usecols=(2,3), unpack=True, skip_footer=15)
#m unit
Grid_size = model.inputDict['GRIDSPC']
Grid_H = model.inputDict['GRIDH'][1]
Highrez_V = model.inputDict['GRIDV'][1]
extend_V = model.inputDict['GRIDV'][0]
Target_depth =  model.inputDict['LAYPOS']
Grid_V = Target_depth - extend_V

if 'OBJRESV' in model.inputDict:
  print("OBJRESV exists in model.inputDict")
  CPPR    = model.inputDict['OBJRESV']
else:
  print("OBJRESV does not exist in model.inputDict")
  CPPR    = model.inputDict['OBJRESH']

vimp    = model.inputDict['OBJVEL']
Rp      = Grid_size*CPPR
vp      = -vimp
ts      = 2.*Rp/vp
num_impactor = model.tru[0].end
num_total = model.tracer_num
dt      = -model.inputDict['DTSAVE']*ts
TEND    = -model.inputDict['TEND']
DTSAVE  = -model.inputDict['DTSAVE']
STEP_num = TEND/DTSAVE
ts_step  = 1/DTSAVE


# fig = plt.figure(figsize=(10,8))
# ax4 = fig.add_subplot(111)

#read No. 1
step0   = model.readStep(['TrP','TrT'],0)
step1   = model.readStep(['TrP','TrT','Trd','Dam'],model.laststep)

x1 = np.where(step0.xmark <= Grid_size*1e3/2)
x2 = x1[0]
x2_1 = x2[x2  > num_impactor ]
x3 = sorted(x2_1,reverse=True)

x3x = step0.xmark[x3]
x3y = -step0.ymark[x3]
x3P = step1.TrP[x3]*1e-9
maxp1 = max(x3P)
maxP1 =round(maxp1,1)
d_r=x3y/(Rp*1e3)

x4 = np.where(step0.xmark <= Grid_size/2*1e3)
x5 = x4[0]
x6 = x5[x5  < num_impactor ]
x6x = step0.xmark[x6]
x6y = step0.ymark[x6]
x6P = step1.TrP[x6]*1e-9
x6T = step1.TrT[x6]
maxp61 = max(x6P[0:20])
maxt61 = max(x6T[0:20])
maxP61 =round(maxp61,1)
maxT61 =round(maxt61,0)
maxp62 = max(x6P)
maxt62 = max(x6T)
maxP62 =round(maxp62,1)
maxT62 =round(maxt62,0)

#cb1.remove()
#ax2.remove(

x4 = np.where(Grid_size*1e3/2 <= step0.xmark)
x5 = x4[0]
x6 = x5[num_impactor <= x5 ]
x6x = step0.xmark[x6]
x6y = -step0.ymark[x6]
x6P = step1.TrP[x6]
x6d = step1.Trd[x6]
maxp3 = max(x6P)

y_g = len(x6)/(Grid_H)
x_g = len(x6)/(Grid_V-1)

LE_shear = 0
LE_comp = 0
LE_tensile = 0
LE_ave = 0
LE_iso_2 = 0
pre_iso = []
x10 = []
x11 = []
thre_ids = []
Grid_A = Grid_size**2

for i in np.arange(0,x_g,1):
  x7 = x6[step0.xmark[x6] <= (i+1)*Grid_size*1e3]
  x8 = x7[i*Grid_size*1e3<= step0.xmark[x7] ]
  x9 = x8
  x9P = step1.TrP[x9]
  # print(LE_shear)
  for j in np.arange(0,len(x9)-1,1):
    if x9P[j] >= 38e+6:
      LE_i = x9P[j]*Grid_A*2*np.pi*((i+1/2)*Grid_size)
      LE_shear = LE_shear + LE_i

for u in np.arange(0,len(x6),1): #loop over tracer clouds
  if x6P[u] >= 38e+6:
      thre_id = num_impactor + u
      x10.append(thre_id)

Region = model.inputDict['GRIDV']
Region_num = Region[0] + Region[1]
Target = model.inputDict['GRIDH']
Target_num = Target[1]+Target[2]

Dam_x = []
Dam_y = []

# for i in np.arange(0,Target_num,1):
#   for j in np.arange(0,Region_num,1):
#     if step1.Dam[i,j] >= 0.5:
#       tracer_id = step1.findTracer(model.x[i,j],model.y[i,j])
#       if isinstance(tracer_id, list):
#         tracer_id = tracer_id[0]
#       if tracer_id >= num_impactor:
#         Dam_x.append(step1.xmark[tracer_id])
#         Dam_y.append(step1.ymark[tracer_id])
#       break


    # if x9P[j] <= 38e+6 and x9P[j+1] >= 38e+6:
    #   thre_id = num_impactor + i*y_g + j
    #   x10.append(thre_id)
    #   print('bottom',thre_id,i,j,step0.xmark[thre_id],step0.ymark[thre_id])
    # if x9P[j] >= 38e+6 and x9P[j+1] <= 38e+6:
    #   thre_id3 = num_impactor + i*y_g + j
    #   x10.append(thre_id)
    #   print('top',thre_id,i,j,step0.xmark[thre_id],step0.ymark[thre_id])

# Plot the pressure field
# for u in range(model.tracer_numu): #loop over tracer clouds
for u in range(1): #loop over tracer clouds
         tstart = model.tru[u].start
         tend = model.tru[u].end
         scat1 = ax5.scatter(step0.xmark[tstart:tend],step0.ymark[tstart:tend],
                c=step1.TrP[tstart:tend]*1e-6,vmin=10,vmax=5e+2,
                 cmap='viridis',s=40,linewidths=0)


cbar = fig.colorbar(scat1,ax=[ax4,ax5], orientation='vertical', pad=0.02)
cbar.set_label('Effective Excavation Region Pressure [MPa]')
ax5.set_xlabel('x  [m]')
ax5.set_ylabel('z  [m]')
ax5.set_xlim([0,75])
ax5.set_ylim([-60,15])

ax5.scatter(step1.xmark[x6],step1.ymark[x6],c='lightblue',s=40,linewidths=0)

ax5.scatter(step0.xmark[x10],step0.ymark[x10],
                c=step1.TrP[x10]*1e-6,vmin=10,vmax=1e+3,
                 cmap='viridis',s=40,linewidths=0)
ax5.axhline(0, color = "black", linewidth=0.8)


# ax4.pcolormesh(model.x,model.y,step1.Dam,vmin=0.13,vmax=1)

#fig.tight_layout(rect=[0,0,0.98,0.9])
# Save the figure
cwd = os.getcwd()
dirname = os.path.basename(cwd)

plt.savefig('../property_iso/{}isobaric_2.png'.format(dirname))